In [1]:
import nengo
import nengo_spa as spa
import pytry
import numpy as np
from random import shuffle
import random
import sys, os
import math
from IPython import display
from scipy.optimize import brute, minimize
import pandas as pd


# Import our classes
sys.path.append('..')
import experiments as xps
from model import ExperimentRun
from data import Data

def create_vocab(D, seed):
    vocab = spa.Vocabulary(dimensions=D, pointer_gen=np.random.RandomState(seed))
    digits = ['D'+str(i+1) for i in range(10)]
    r = np.random.RandomState(seed)
    r.shuffle(digits) # avoid any bias in similarity
    vocab.populate(';'.join(digits))
    keys = ['FIXATE', 'MASK', \
               'MORE','LESS', \
               'GET','SET', \
    'SIMPLE', 'CHAINED_ADD', 'CHAINED_SUB']
    vocab.populate(";".join(keys))
    
    vocab.populate('SOURCE ; CONTENT ;'+
                      'V ; COM ; ADD ; SUB ; M')
    vocab.populate(";".join([p+"_SOURCE=(SOURCE*"+p+").normalized()" for p in ['V', 'COM', 'ADD', 'SUB']])) # this is done to avoid similarity with other SPs
    return vocab

def create_xp(n_blocks_per_operation, n_trials_per_digit, n_different_digits, n_different_operations, stim_duration):
    trials = xps.createTrials(n_blocks_per_operation, n_trials_per_digit, n_different_digits, n_different_operations, shuffle=True)
    return xps.Xp1(0, trials, stim_duration=stim_duration)

# Optimization and model parameters

In [2]:
D = int(192)  # the dimensionality of the vectors
PROC_FDBCK = .5
GW_FDBCK = .95 # but synapse is shorter than for perfect storage
GW_SCALE = 20 # scale the input of GW for saturation and catching very short visual stimuli
BG_THR = .1
BG_BIAS = .5
STIM_DURATION = .1
N_NEURONS_SCALE = 1
N_SEEDS = 5
STARTING_SEED = 1

N_BLOCKS_PER_OPERATION = 1 # default: 10
N_TRIALS_PER_DIGIT = 2 # default: 5
N_DIFFERENT_DIGITS = 4 # default: 4
N_DIFFERENT_OPERATIONS = 3 # default: 3


In [3]:
def evaluation_function(params):
    
    print("params:", params)
    data = []

    
    print('Simulating...')
    for i,seed in enumerate(range(STARTING_SEED, STARTING_SEED+N_SEEDS)):
        print('\t seed '+str(i+1)+'/'+str(N_SEEDS))
        data_filename = 'run_seed'+str(seed)+'_scross'+str(params[0])+'_sevid'+str(params[1])#+'_tsenso'+str(params[2])
        
        xp = create_xp(N_BLOCKS_PER_OPERATION, N_TRIALS_PER_DIGIT, N_DIFFERENT_DIGITS, N_DIFFERENT_OPERATIONS, STIM_DURATION)
        results = ExperimentRun().run(
            n_neurons_scale=N_NEURONS_SCALE,
            s_crosstalk=params[0],
            s_evidence=params[1],
            t_senso=.175,#params[2],
            vocab=create_vocab(D, seed),
            xp=xp,
            proc_feedback=PROC_FDBCK,
            GW_feedback=GW_FDBCK,
            GW_scale=GW_SCALE,
            BG_thr=BG_THR,
            BG_bias=BG_BIAS,
            seed=seed,
            backend="nengo_ocl",
            data_filename=data_filename
        )
        
        results['data'].df.to_pickle('./data/'+data_filename+'.csv')
        data.append(results['data'].df)
        
        
        
    data = Data(pd.concat(data))
    fitness_error = data.fitness_error
    print('error:', fitness_error)
    return fitness_error
    


In [ ]:
x0, fval, grid, jout = brute(
    evaluation_function, 
    (
        (0, 1.5), # bound for s_crosstalk
        (0, 15), # bound for s_evidence
        #(0,.5), # bound for t_senso (in seconds)
    ),
    Ns=5, 
    full_output=True
)

params: [0. 0.]
Simulating...
	 seed 1/5
running run_seed1_scross0.0_sevid0.0
No context argument was provided to nengo_ocl.Simulator
Calling pyopencl.create_some_context() for you now:


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

seed = 1
backend = 'nengo_ocl'
dt = 0.001
neuron_type = 'default'
n_neurons_scale = 1
s_crosstalk = 0.0
s_evidence = 0.0
t_senso = 0.175
vocab = <nengo_spa.vocabulary.Vocabulary object at 0x7f31ebefe290>
xp = <experiments.Xp1 object at 0x7f31ebf1fc90>
proc_feedback = 0.5
GW_feedback = 0.95
GW_scale = 20
BG_thr = 0.1
BG_bias = 0.5
data = <data.Data object at 0x7f31e1ee5910>
	 seed 2/5
running run_seed2_scross0.0_sevid0.0


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

...

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

seed = 3
backend = 'nengo_ocl'
dt = 0.001
neuron_type = 'default'
n_neurons_scale = 1
s_crosstalk = 1.5
s_evidence = 3.75
t_senso = 0.175
vocab = <nengo_spa.vocabulary.Vocabulary object at 0x7f31bc7e13d0>
xp = <experiments.Xp1 object at 0x7f31e1de83d0>
proc_feedback = 0.5
GW_feedback = 0.95
GW_scale = 20
BG_thr = 0.1
BG_bias = 0.5
data = <data.Data object at 0x7f31983503d0>
	 seed 4/5
running run_seed4_scross1.5_sevid3.75


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

In [ ]:
print(x0, fval, grid, jout)
np.save("./data/x0", x0)
np.save("./data/fval", fval)
np.save("./data/grid", grid)
np.save("./data/jout", jout)